In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

from function import log
import config


import pandas_gbq
import google

from google.oauth2 import service_account


In [2]:
# access garmin and authentificate

s=Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=s)
driver.get("https://sso.garmin.com/sso/login?service=https%3A%2F%2Fconnect.garmin.com%2Fpost-auth%2Flogin&webhost=olaxpw-connect04&source=https%3A%2F%2Fconnect.garmin.com%2Fen-US%2Fsignin&redirectAfterAccountLoginUrl=https%3A%2F%2Fconnect.garmin.com%2Fpost-auth%2Flogin&redirectAfterAccountCreationUrl=https%3A%2F%2Fconnect.garmin.com%2Fpost-auth%2Flogin&gauthHost=https%3A%2F%2Fsso.garmin.com%2Fsso&locale=en_US&id=gauth-widget&cssUrl=https%3A%2F%2Fstatic.garmincdn.com%2Fcom.garmin.connect%2Fui%2Fcss%2Fgauth-custom-v1.1-min.css&clientId=GarminConnect&rememberMeShown=true&rememberMeChecked=false&createAccountShown=true&openCreateAccount=false&usernameShown=false&displayNameShown=false&consumeServiceTicket=false&initialFocus=true&embedWidget=false&generateExtraServiceTicket=false#")

#wait until sign-in fields are visible
#wait = WebDriverWait(driver, 60)
#wait.until(EC.frame_to_be_available_and_switch_to_it(("id","gauth-widget-frame-gauth-widget")))
#wait.until(EC.presence_of_element_located(("id","username")))

#write login info to fields, then submit
print("Signing in to connect.garmin.com")
element = driver.find_element_by_id("username")
element.send_keys(config.USER_EMAIL)
element = driver.find_element_by_id("password")
element.send_keys(config.USER_PASS)
element.send_keys(Keys.RETURN)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/florianniclause/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


Signing in to connect.garmin.com


<ipython-input-2-dce7bb299659>:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("username")
<ipython-input-2-dce7bb299659>:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("password")


In [3]:
health_df=pd.read_csv('health.csv')

In [4]:
from datetime import datetime
datelist = pd.date_range(start='2021-07-01',end='2021-09-30'#datetime.today().strftime('%Y-%m-%d')) 
                        )
pd_date = pd.DataFrame(datelist.astype('str'),columns=['date'])

pd_date

,date
0,2021-07-01
1,2021-07-02
2,2021-07-03
3,2021-07-04
4,2021-07-05
...,...
87,2021-09-26
88,2021-09-27
89,2021-09-28
90,2021-09-29


In [5]:
list_to_update=pd.merge(health_df,pd_date,how='right',on='date')
list_to_update

,date,score_sleep,Duree_sommeil,coucher,lever,FC_repos,FC_max_day,stress,training_status,training_comment,VO2_max_run,training_load
0,2021-07-01,NaN,NaN,NaN,NaN,84.0,199.0,28.0,Productif,Manque de charge d'aérobie faible,50.0,"1,617"
1,2021-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non productif,Manque de charge d'aérobie faible,50.0,"1,735"
2,2021-07-03,NaN,NaN,NaN,NaN,NaN,NaN,26.0,Non productif,Manque de charge d'aérobie faible,50.0,"1,776"
3,2021-07-04,NaN,NaN,NaN,NaN,NaN,NaN,11.0,Effort trop soutenu,Manque de charge d'aérobie faible,50.0,"1,979"
4,2021-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non productif,Manque de charge d'aérobie faible,50.0,"1,747"
...,...,...,...,...,...,...,...,...,...,...,...,...
87,2021-09-26,NaN,NaN,NaN,NaN,NaN,NaN,79.0,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"3,823"
88,2021-09-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"3,082"
89,2021-09-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"1,317"
90,2021-09-29,NaN,NaN,NaN,NaN,NaN,NaN,6.0,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"1,942"


In [6]:
list_to_update=list_to_update[list_to_update.isna().any(axis=1)]

#add today's date
today_=[[np.nan] * (len(list_to_update.columns))]
today_[0][0]=str(datetime.today().strftime('%Y-%m-%d'))
df1=pd.DataFrame(today_, columns=list_to_update.columns)
list_to_update = df1.append(list_to_update, ignore_index=True)


list_to_update

,date,score_sleep,Duree_sommeil,coucher,lever,FC_repos,FC_max_day,stress,training_status,training_comment,VO2_max_run,training_load
0,2022-06-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-07-01,NaN,NaN,NaN,NaN,84.0,199.0,28.0,Productif,Manque de charge d'aérobie faible,50.0,"1,617"
2,2021-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non productif,Manque de charge d'aérobie faible,50.0,"1,735"
3,2021-07-03,NaN,NaN,NaN,NaN,NaN,NaN,26.0,Non productif,Manque de charge d'aérobie faible,50.0,"1,776"
4,2021-07-04,NaN,NaN,NaN,NaN,NaN,NaN,11.0,Effort trop soutenu,Manque de charge d'aérobie faible,50.0,"1,979"
...,...,...,...,...,...,...,...,...,...,...,...,...
88,2021-09-26,NaN,NaN,NaN,NaN,NaN,NaN,79.0,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"3,823"
89,2021-09-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"3,082"
90,2021-09-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"1,317"
91,2021-09-29,NaN,NaN,NaN,NaN,NaN,NaN,6.0,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"1,942"


In [7]:
for i in range(0,list_to_update.shape[0]):
    driver.get(f"https://connect.garmin.com/modern/sleep/{list_to_update.iloc[i]['date']}/score")
    time.sleep(5)
    try :
        print(driver.find_element_by_xpath('//div[contains(@class, "SleepScoreSummary_sleepScoreNumber")]').text)
        list_to_update.at[i,'score_sleep'] = driver.find_element_by_xpath('//div[contains(@class, "SleepScoreSummary_sleepScoreNumber")]').text
    except:
        pass
    driver.get(f"https://connect.garmin.com/modern/sleep/{list_to_update.iloc[i]['date']}")
    time.sleep(5)
    try:
        sleep_hour=driver.find_element_by_xpath("//div[contains(@class, 'SleepTimeEditor')]").text.split('\n')
        list_to_update.at[i,'coucher']=sleep_hour[0]
        list_to_update.at[i,'lever']=sleep_hour[2]
    except:
        pass
    try:
        duree=driver.find_element_by_xpath("//div[contains(@class, 'sleepGaugeContainer')]").text.replace('h','').replace('m','').split('\n')[0].split(' ')
        list_to_update.at[i,'Duree_sommeil']=int(duree[0])*60+int(duree[1])
    except:
        pass
    driver.get(f"https://connect.garmin.com/modern/daily-summary/{list_to_update.iloc[i]['date']}/heartRate")
    time.sleep(10)
    try:
        hr=driver.find_element_by_xpath("//div[contains(@class, 'HeartRateCardMain')]").text.split('\n')
        list_to_update.at[i,'FC_repos']=hr[2].replace(' bpm','')
        list_to_update.at[i,'FC_max_day']=hr[4].replace(' bpm','')
    except:
        pass
    try:
        list_to_update.at[i,'stress']=driver.find_element_by_xpath("//div[contains(@class, 'StressCardMain')]").text.split('\n')[0]
    except:

        pass
    try:
        training_status=driver.find_element_by_xpath("//div[contains(@class, 'TrainingStatus')]").text.split('\n')
        list_to_update.at[i,'training_status']=training_status[0]
        list_to_update.at[i,'training_comment']=training_status[1]
        list_to_update.at[i,'VO2_max_run']=training_status[3]
        list_to_update.at[i,'training_load']=training_status[5]
    except:
        pass
list_to_update

<ipython-input-7-7d3140161b73>:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  print(driver.find_element_by_xpath('//div[contains(@class, "SleepScoreSummary_sleepScoreNumber")]').text)
<ipython-input-7-7d3140161b73>:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  list_to_update.at[i,'score_sleep'] = driver.find_element_by_xpath('//div[contains(@class, "SleepScoreSummary_sleepScoreNumber")]').text


40


<ipython-input-7-7d3140161b73>:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  sleep_hour=driver.find_element_by_xpath("//div[contains(@class, 'SleepTimeEditor')]").text.split('\n')
<ipython-input-7-7d3140161b73>:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  duree=driver.find_element_by_xpath("//div[contains(@class, 'sleepGaugeContainer')]").text.replace('h','').replace('m','').split('\n')[0].split(' ')
<ipython-input-7-7d3140161b73>:25: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  hr=driver.find_element_by_xpath("//div[contains(@class, 'HeartRateCardMain')]").text.split('\n')
<ipython-input-7-7d3140161b73>:31: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  list_to_update.at[i,'stress']=driver.find_el

,date,score_sleep,Duree_sommeil,coucher,lever,FC_repos,FC_max_day,stress,training_status,training_comment,VO2_max_run,training_load
0,2022-06-17,40,247,0:25,6:02,54.0,89.0,24.0,Productif,Manque d'anaérobie,56.0,"1,292"
1,2021-07-01,NaN,NaN,NaN,NaN,84.0,199.0,28.0,Productif,Manque de charge d'aérobie faible,50.0,"1,617"
2,2021-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non productif,Manque de charge d'aérobie faible,50.0,"1,735"
3,2021-07-03,NaN,NaN,NaN,NaN,NaN,NaN,26.0,Non productif,Manque de charge d'aérobie faible,50.0,"1,776"
4,2021-07-04,NaN,NaN,NaN,NaN,NaN,NaN,11.0,Effort trop soutenu,Manque de charge d'aérobie faible,50.0,"1,979"
...,...,...,...,...,...,...,...,...,...,...,...,...
88,2021-09-26,NaN,NaN,NaN,NaN,NaN,NaN,79.0,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"3,823"
89,2021-09-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"3,082"
90,2021-09-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"1,317"
91,2021-09-29,NaN,NaN,NaN,NaN,NaN,NaN,6.0,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"1,942"


In [8]:
list_to_update

,date,score_sleep,Duree_sommeil,coucher,lever,FC_repos,FC_max_day,stress,training_status,training_comment,VO2_max_run,training_load
0,2022-06-17,40,247,0:25,6:02,54.0,89.0,24.0,Productif,Manque d'anaérobie,56.0,"1,292"
1,2021-07-01,NaN,NaN,NaN,NaN,84.0,199.0,28.0,Productif,Manque de charge d'aérobie faible,50.0,"1,617"
2,2021-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non productif,Manque de charge d'aérobie faible,50.0,"1,735"
3,2021-07-03,NaN,NaN,NaN,NaN,NaN,NaN,26.0,Non productif,Manque de charge d'aérobie faible,50.0,"1,776"
4,2021-07-04,NaN,NaN,NaN,NaN,NaN,NaN,11.0,Effort trop soutenu,Manque de charge d'aérobie faible,50.0,"1,979"
...,...,...,...,...,...,...,...,...,...,...,...,...
88,2021-09-26,NaN,NaN,NaN,NaN,NaN,NaN,79.0,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"3,823"
89,2021-09-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"3,082"
90,2021-09-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"1,317"
91,2021-09-29,NaN,NaN,NaN,NaN,NaN,NaN,6.0,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"1,942"


In [9]:
#suppress duplicate
list_to_add = list_to_update.dropna(how='all')['date']

health_df = health_df[~health_df['date'].isin(list_to_add)]

#append list_to_update to health_df
health_df.append(list_to_update)



,date,score_sleep,Duree_sommeil,coucher,lever,FC_repos,FC_max_day,stress,training_status,training_comment,VO2_max_run,training_load
1,2022-06-16,NaN,NaN,NaN,NaN,61.0,199.0,48.0,Productif,Manque d'anaérobie,56.0,"1,578"
2,2022-06-15,38,337,23:40,5:48,58.0,177.0,43.0,Productif,Manque d'anaérobie,56.0,"1,549"
3,2022-06-14,No data,No data,No data,No data,51.0,180.0,52.0,Productif,Manque d'anaérobie,56.0,"1,565"
4,2022-06-13,85,425,22:50,6:25,42.0,167.0,34.0,Productif,Manque d'anaérobie,56.0,"1,572"
5,2022-06-12,29,229,1:06,5:05,45.0,111.0,31.0,Productif,Manque d'anaérobie,56.0,"1,629"
...,...,...,...,...,...,...,...,...,...,...,...,...
88,2021-09-26,NaN,NaN,NaN,NaN,NaN,NaN,79.0,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"3,823"
89,2021-09-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"3,082"
90,2021-09-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"1,317"
91,2021-09-29,NaN,NaN,NaN,NaN,NaN,NaN,6.0,Effort trop soutenu,Manque de charge d'aérobie faible,48.0,"1,942"


In [10]:
#append list_to_update to health_df
health_df = health_df.append(list_to_update).sort_values('date',ascending=False).drop_duplicates()

In [11]:
health_df.to_csv('health.csv', index=False)

In [12]:
health_df

,date,score_sleep,Duree_sommeil,coucher,lever,FC_repos,FC_max_day,stress,training_status,training_comment,VO2_max_run,training_load
0,2022-06-17,40,247,0:25,6:02,54.0,89.0,24.0,Productif,Manque d'anaérobie,56.0,"1,292"
1,2022-06-16,NaN,NaN,NaN,NaN,61.0,199.0,48.0,Productif,Manque d'anaérobie,56.0,"1,578"
2,2022-06-15,38,337,23:40,5:48,58.0,177.0,43.0,Productif,Manque d'anaérobie,56.0,"1,549"
3,2022-06-14,No data,No data,No data,No data,51.0,180.0,52.0,Productif,Manque d'anaérobie,56.0,"1,565"
4,2022-06-13,85,425,22:50,6:25,42.0,167.0,34.0,Productif,Manque d'anaérobie,56.0,"1,572"
...,...,...,...,...,...,...,...,...,...,...,...,...
5,2021-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non productif,Manque de charge d'aérobie faible,50.0,"1,747"
4,2021-07-04,NaN,NaN,NaN,NaN,NaN,NaN,11.0,Effort trop soutenu,Manque de charge d'aérobie faible,50.0,"1,979"
3,2021-07-03,NaN,NaN,NaN,NaN,NaN,NaN,26.0,Non productif,Manque de charge d'aérobie faible,50.0,"1,776"
2,2021-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non productif,Manque de charge d'aérobie faible,50.0,"1,735"


In [13]:
import numpy
health_df[health_df=='No data']=numpy.nan
health_df[health_df=='--']=numpy.nan

In [14]:
health_df.score_sleep=health_df.score_sleep.astype('float')
health_df.Duree_sommeil=health_df.Duree_sommeil.astype('float')
health_df.FC_repos=health_df.FC_repos.astype('float')
health_df.FC_max_day=health_df.FC_max_day.astype('float')
health_df.stress=health_df.stress.astype('float')
health_df.VO2_max_run=health_df.VO2_max_run.astype('float')
#health_df.training_load=health_df.training_load.astype('float')

In [15]:
def date_function(date_):
    return datetime.fromisoformat(date_)
def get_weekday(date_):
    #date_=datetime.fromisoformat(date_[:-1] + '+00:00').astimezone(timezone.utc)
    return calendar.day_name[date_.weekday()]
def get_day(date_):
    return date_.day
def get_month(date_):
    return date_.month
def get_year(date_):
    return date_.year
def get_week(date_):
    return date_.week

def localize_time(time_):
    return time_.tz_localize('Europe/Paris')

In [16]:
datetime.fromisoformat(health_df['date'].iloc[0])

datetime.datetime(2022, 6, 17, 0, 0)

In [17]:
health_df['date']=health_df['date'].apply(date_function)

In [18]:
#health_df['weekday']=health_df['start_date_local'].apply(get_weekday)
#health_df['day']=health_df['start_date_local'].apply(get_day)
health_df['week']=health_df['date'].apply(get_week)
#health_df['month']=health_df['start_date_local'].apply(get_month)
health_df['year']=health_df['date'].apply(get_year)

In [19]:
health_w=health_df.groupby(['week','year'],as_index=False).agg({'score_sleep': ['mean'],
                                                        'Duree_sommeil': ['mean'],
                                                       'FC_repos': ['mean'],
                                                       'FC_max_day': ['max'],
                                                       'stress': ['mean'],
                                                        #'training_status': lambda x:x.value_counts().index[0],
                                                         #'training_comment': lambda x:x.value_counts().index[0],
                                                         #'VO2_max_run': ['mean'],
                                                         #'training_load': ['mean'],
                                                        }).sort_values(by=['year','week'],ascending=[False,False]).round(1)

In [20]:
columns = health_w.columns.tolist()
health_w.columns = ['_'.join(tuple(reversed(list(col)))) for col in columns]

In [21]:
health_w.to_csv('health_w.csv',index=False)

In [22]:
health_w

,_week,_year,mean_score_sleep,mean_Duree_sommeil,mean_FC_repos,max_FC_max_day,mean_stress
51,52,2022,52.0,370.5,51.0,109.0,31.5
23,24,2022,54.3,336.3,53.2,199.0,40.2
22,23,2022,50.0,383.3,46.7,207.0,35.6
21,22,2022,57.8,381.0,47.9,185.0,33.0
20,21,2022,61.4,389.6,46.9,193.0,26.7
19,20,2022,53.7,357.7,50.9,204.0,36.3
18,19,2022,55.1,379.6,50.1,189.0,32.9
17,18,2022,51.3,353.4,50.3,185.0,30.7
16,17,2022,42.1,377.9,54.1,189.0,35.9
15,16,2022,72.2,398.5,45.7,189.0,28.1
